In [56]:
import sys
sys.path.append('/home/cryptobrahman/Own/football_competitions_research/own_modules/') 
sys.path.append('/mnt/KINGSTON_120/Own/football_competitions_research/own_modules/') 

In [57]:
import pickle
import pandas as pd
from pprint import pprint

from _astro_constants import AspectsCalculate, AstrologicalPoints, ObjectsPrepare, AstrologicalConstants, TransformDoubleValues, AspectsPrepare, AspectsClearing

pd.set_option('display.max_columns', 200)  # or 1000
pd.set_option('display.max_rows', 200)  # or 1000
pd.set_option('display.max_colwidth', 100)  # or 199

In [58]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [59]:
# data = {'date':['2015/03/13', '2016/05/23', '2018/10/03'], 'utc_time':['14:00', '12:30', '00:15'], 'lon':['38n32', '08n07', '14s42'], 'lat':['8w54', '18e14','36w04']}
# df = pd.DataFrame(data)
# df

In [60]:
with open('pickle_files/cities_countries/df_with_geo_coords_26_05_2022', 'rb') as f:
    df_coords = pickle.load(f)

In [61]:
print('shape - ', df_coords.shape)
df_coords.head()

shape -  (8973, 25)


,game_id,ligue_header,comp_id,season_id,game_utc,game_title,goals,ligue,f_team,pos_f,pts_f,pos_s,pts_s,fs_pos_max,fs_pts_max,f_pow,desc,country,team_id,city,iso_code,longitude,latitude,lat,lon
0,1708335,Premier League,483,301,26.05.2022 15:00,FC Minsk - Arsenal Dzerzhinsk,2:0,top,FC Minsk,4.0,19.0,15.0,4.0,"4 - 15, 16","19 - 4, 23",Fav,fav_pre,Belarus,540,Minsk,BY,27.56667,53.90000,53n53,27e34
1,1687696,OBOS liga,668,301,26.05.2022 15:00,Skeid - Sandnes Ulf,0:1,thr,Skeid,16.0,3.0,4.0,18.0,"16 - 4, 16","3 - 18, 23",Pre,fav_pre,Norway,10294,Oslo,NO,10.74609,59.91273,59n54,10e44
2,1687694,OBOS liga,668,301,26.05.2022 16:00,Kongsvinger - Brann,1:3,thr,Kongsvinger,13.0,7.0,1.0,23.0,"13 - 1, 16","7 - 23, 23",Pre,fav_pre,Norway,None,Kongsvinger,NO,11.99772,60.19049,60n11,11e59
3,1687692,OBOS liga,668,301,26.05.2022 16:00,Bryne - Stabaek,0:0,thr,Bryne,12.0,8.0,3.0,18.0,"12 - 3, 16","8 - 18, 23",Pre,fav_pre,Norway,None,Bryne,NO,5.64766,58.73536,58n44,5e38
4,1687698,OBOS liga,668,301,26.05.2022 16:00,Stjordals-Blink - Sogndal,1:3,thr,Stjordals-Blink,15.0,5.0,8.0,14.0,"15 - 8, 16","5 - 14, 23",Pre,fav_pre,Norway,8922,Stjørdalshalsen,NO,10.92618,63.46810,63n28,10e55


#### Check data for geocoords on google maps. Exampl lat-lon: 69.245016 17.952091

In [62]:
print('https://www.google.com.ua/maps/')

https://www.google.com.ua/maps/


In [63]:
df_coords.loc[200:200, ['city', 'country', 'latitude', 'longitude']]

,city,country,latitude,longitude
200,Berlin,Germany,52.52437,13.41053


In [64]:
df             = df_coords[['game_utc', 'lon', 'lat']].copy()
df['date']     = df.game_utc.str[:10].str.split('.')
df['utc_time'] = df.game_utc.str[11:16]
df.date        = df.date.str[2] +'/'+ df.date.str[1] +'/'+ df.date.str[0]

In [65]:
df.drop(columns='game_utc', inplace=True)

In [66]:
df = df[['date', 'utc_time', 'lon', 'lat']]

In [67]:
print('shape - ', df.shape)
df.tail()

shape -  (8973, 4)


,date,utc_time,lon,lat
8968,2021/03/30,18:45,4w15,55n51
8969,2021/03/30,18:45,4w1,55n51
8970,2021/03/30,18:45,2w39,56n43
8971,2021/03/30,18:45,3w20,56n6
8972,2021/03/29,19:00,5e18,51n41


### Get astrological objects

In [68]:
df[(df.lat.isna()) | (df.lon.isna())]

,date,utc_time,lon,lat


In [69]:
# df_coords.loc[44:44, ['country', 'city', 'longitude', 'latitude']]

In [70]:
# df.loc[44:44,]

In [71]:
# import datetime
# from flatlib import ephem, const

# fmt = '%Y/%m/%d %H:%M'
# s   = '2022/05/23 12:00'
# dt  = datetime.datetime.strptime(s, fmt)
# tt  = dt.timetuple()
# jd  = tt.tm_year * 1000 + tt.tm_yday
# print(jd)
# lat = 40.245016
# lon = 17.952091
# ephem.swe.sweHouses(jd, lat, lon, hsys=const.HOUSES_PLACIDUS)

In [72]:
df_dates_posits = AstrologicalPoints.dates_posits_calculate(df, date='date', utc_time='utc_time', lat='lat', lon='lon')

In [73]:
print('shape - ', df_dates_posits.shape)
df_dates_posits.head(3)

shape -  (8973, 2)


,dates,posits
0,<2022/05/26 15:00:00 00:00:00>,<53N53:00 27E34:00>
1,<2022/05/26 15:00:00 00:00:00>,<59N54:00 10E44:00>
2,<2022/05/26 16:00:00 00:00:00>,<60N11:00 11E59:00>


#### Check rows with errors for chart calculate (north cities houses calc errors)

In [74]:
error_inds = []

for i in range(df_dates_posits.shape[0]):
    try:
        AstrologicalPoints.charts_calculate(df_dates_posits.loc[i:i,])
    except Exception as e: 
        error_inds.append(i)

In [75]:
print('len - ', len(error_inds))
error_inds[:3]

len -  26


[44, 939, 3662]

In [76]:
df_dates_posits.drop(index=error_inds, inplace=True)

In [77]:
print('shape - df              - ', df.shape)
print('shape - df_dates_posits - ', df_dates_posits.shape)
df_dates_posits.tail(3)

shape - df              -  (8973, 4)
shape - df_dates_posits -  (8947, 2)


,dates,posits
8970,<2021/03/30 18:45:00 00:00:00>,<56N43:00 02W39:00>
8971,<2021/03/30 18:45:00 00:00:00>,<56N06:00 03W20:00>
8972,<2021/03/29 19:00:00 00:00:00>,<51N41:00 05E18:00>


In [78]:
sr_charts = AstrologicalPoints.charts_calculate(df_dates_posits)

In [79]:
print('shape - df_coords - ', df_coords.shape)
df_coords.drop(index=error_inds, inplace=True)
df_coords.reset_index(drop=True, inplace=True)
print('shape - df_coords - ', df_coords.shape)

shape - df_coords -  (8973, 25)
shape - df_coords -  (8947, 25)


In [80]:
df_coords['chart'] = sr_charts

In [81]:
df_coords.head(3)

,game_id,ligue_header,comp_id,season_id,game_utc,game_title,goals,ligue,f_team,pos_f,pts_f,pos_s,pts_s,fs_pos_max,fs_pts_max,f_pow,desc,country,team_id,city,iso_code,longitude,latitude,lat,lon,chart
0,1708335,Premier League,483,301,26.05.2022 15:00,FC Minsk - Arsenal Dzerzhinsk,2:0,top,FC Minsk,4.0,19.0,15.0,4.0,"4 - 15, 16","19 - 4, 23",Fav,fav_pre,Belarus,540,Minsk,BY,27.56667,53.90000,53n53,27e34,<flatlib.chart.Chart object at 0x7f4bcb759580>
1,1687696,OBOS liga,668,301,26.05.2022 15:00,Skeid - Sandnes Ulf,0:1,thr,Skeid,16.0,3.0,4.0,18.0,"16 - 4, 16","3 - 18, 23",Pre,fav_pre,Norway,10294,Oslo,NO,10.74609,59.91273,59n54,10e44,<flatlib.chart.Chart object at 0x7f4bcb7594c0>
2,1687694,OBOS liga,668,301,26.05.2022 16:00,Kongsvinger - Brann,1:3,thr,Kongsvinger,13.0,7.0,1.0,23.0,"13 - 1, 16","7 - 23, 23",Pre,fav_pre,Norway,None,Kongsvinger,NO,11.99772,60.19049,60n11,11e59,<flatlib.chart.Chart object at 0x7f4bcb759c70>


In [82]:
df_coords.rename(columns={'chart':'transform_id'}, inplace=True)

In [83]:
obj = df_coords.transform_id[0]

In [84]:
AstrologicalPoints.AstrologicalPoints()

SyntaxError: invalid syntax (4052849050.py, line 1)

In [ ]:
for :
    print(obj.)

In [ ]:
# objects = ObjectsPrepare(df_coords).conbine_class_methods()

### Calculate aspects

In [ ]:
def aspects_calculate(df: pd.DataFrame, row_number: int):
    objects = ObjectsPrepare(df.iloc[row_number]).conbine_class_methods()
#     tuple_list = df.transform_id[row_number]

#     all_aspects = []

#     houses_remove_objs = []
#     moon_remove_objs   = ['Asc', 'Desc', 'MC', 'IC', 'Uranus', 'Neptune', 'Pluto', 'Chiron', 'North Node', 'South Node', 'Antes Moon','Antes Uranus', 
#                           'Antes Neptune', 'Antes Pluto', 'Antes Chiron']
#     main_remove_objs   = ['Asc', 'Desc', 'MC', 'IC', 'Moon', 'Antes Moon']
#     houses_aspect_degs = {'Con': 0}
#     main_aspect_degs   = {'Con': 0, 'Opp': 180}
#     moon_aspect_degs   = AstrologicalConstants.aspekts_degrees
#     house_ids          = AstrologicalConstants.houses

#     house_objs, moon_objs, main_objs = objects.copy(),objects.copy(), objects.copy() 

#     moon_asp_after  = AspectsPrepare.object_aspects('Moon', moon_objs, moon_remove_objs, moon_aspect_degs)
#     moon_asp_before = AspectsPrepare.object_aspects('Moon', moon_objs, moon_remove_objs, moon_aspect_degs, before_point_asp='yes', before_orb=1)
#     all_aspects.append(moon_asp_after)
#     all_aspects.append(moon_asp_before)

#     for h_obj in house_ids:
#         house_asp_after  = AspectsPrepare.object_aspects(h_obj, house_objs, houses_remove_objs, houses_aspect_degs, before_point_asp='no', after_orb=3)
#         house_asp_before = AspectsPrepare.object_aspects(h_obj, house_objs, houses_remove_objs, houses_aspect_degs, before_point_asp='yes', before_orb=3)
#         all_aspects.append(house_asp_after)
#         all_aspects.append(house_asp_before)

#     for m_obj in objects:
#         main_asp_after  = AspectsPrepare.object_aspects(m_obj.id, main_objs, main_remove_objs, main_aspect_degs, before_point_asp='no', after_orb=3)
#         main_asp_before = AspectsPrepare.object_aspects(m_obj.id, main_objs, main_remove_objs, main_aspect_degs, before_point_asp='yes', before_orb=3)
#         all_aspects.append(main_asp_after)
#         all_aspects.append(main_asp_before)

#     for asp in all_aspects:
#         if asp != None and any(var['f_point'] == 'Asc' and var['s_point'] == 'Pluto' for var in asp):
#             asc_pluto = asp
#         else:  
#             asc_pluto = []

#     if asc_pluto == []:
#         for obj in objects:
#             if obj.id == 'Asc' or obj.id == 'Pluto':
#                 asc_pluto.append(obj) 

#         asc_pluto_asp_after = AspectsPrepare.object_aspects('Asc', asc_pluto, houses_remove_objs, houses_aspect_degs, before_point_asp='no', after_orb=20)
#         all_aspects.append(asc_pluto_asp_after)

#     all_aspects_full = AspectsPrepare.transform_dict_list_type(all_aspects)

#     AspectsClearing.remove_duplicate_aspects(all_aspects_full)
#     AspectsClearing.remove_antes_unimportant_aspects(all_aspects_full)
#     AspectsClearing.remove_node_opposition(all_aspects_full)
#     AspectsClearing.remove_antes_with_own_nat_point_aspects(all_aspects_full)
#     AspectsClearing.remove_mirror_antes_aspects(all_aspects_full, tuple_list)
#     AspectsClearing.remove_mirror_houses_antes_conuction(all_aspects_full)

#     for asp in all_aspects_full:
#         asp['den_point'] = []

#     AspectsPrepare.supplement_houses_and_parts_rulers_aspects(objects, all_aspects_full, tuple_list)
#     AspectsPrepare.supplement_conv_stat_aspects(objects, all_aspects_full)
#     AspectsPrepare.supplement_moon_complete_denide_weak_aspects(objects, all_aspects_full, tuple_list)
#     AspectsPrepare.scale_orb_characteristic(all_aspects_full)
#     AspectsPrepare.moon_scale_orb_characteristic(all_aspects_full)
#     AspectsPrepare.aspects_rulers_characteristic(tuple_list, all_aspects_full)

#     AspectsClearing.remove_houses_one_nodes_conuction(all_aspects_full)  

    return objects#[all_aspects_full]

In [ ]:
# aspects = []
# for x in range(df.shape[0]):
#      aspects.append(aspects_calculate(df.loc[0:0,], x))  

# df['aspects'] = aspects        

In [ ]:
# pprint(aspects, sort_dicts=False)

In [ ]:
# print(len(aspects))
# for x in aspects:
#     print(x)

In [ ]:
# for x in objects:
#     if x.id == 'Mars':
#         x.lon = 1
#     if x.id == 'Pluto':
#         x.lon = 2      

In [ ]:
#     Rules for aspekts:
#     Antes Moon - only for house conuction
#     Antes has only house rulers planets, higher planets and Parse Fortuna.

#     Moon aspects by specific points (houses rulers, there antes and fortuna parts with it antes):
#     'moon_conv_clear_compl' - any first aspect by parse fortuna and it antes, or first 'Con' aspect to houses rulers or any last aspect to houses rulers without 
#       any point aspects beefore
#     'moon_conv_compl' - any first aspect by parse fortuna and it antes, or if first 'Con' aspect to houses rulers or any last aspect to houses rulers with 
#       aspected beefore any point among active planets and only 'Con' type for all other points if their not in house rulers
#     'moon_conv_compl_weak' - any first aspect by parse fortuna and it antes or if first 'Con' aspect to houses rulers or any last houses rulers's aspect 
#       which was aspecting beefore parses or other houses rulers and last aspect antes's houses rulers which wasn't aspecting beefore parses or other houses rulers -------
#     'moon_conv' - all other points aspects
#     'moon_conv_denide' - if aspect sing: 'diff' or if aspect's 'tr_orb' > 5.1 
#     'moon_diver', 'moon_diver_weak' - for divergent Moon aspects.

#     Houses rulers and parts rulers aspects are aspecting own parts(houses):
#     'conv_compl', (conv_in_compl', conv_out_compl') - approuch parts(houses) rulers aspecting own parts(houses) without Moon's aspect at part 
#     'conv_denide', (conv_in_weak', conv_out_weak') - if parts(houses) rulers has other early conversation aspect(Moon include) or locate in different sings
#     'diver_weak', (diver_in_weak', diver_out_weak') - if aspect is divergent
#     'diver_denide' - if pars ruler has divergent aspect and it has previos con aspect

#     Other 'conv' and 'stat' aspects:
#     'conv_denide', 'stat_denide' - if points locate in different sings.
#     'conv_weak', 'stat_weak', ('in_conv_weak', 'out_conv_weak', 'in_stat_weak', 'out_stat_weak') - weakened aspects with the other long point 
#     located beetwen points longs. 

#     Remove antes aspects: 
#     If exist aspect between natural points - remove all unimportant aspects with anteses:
#       Stay - ('Mars' con 'Venus') or ('Mars' opp 'Venus')
#       Remove aspects ('Mars' opp 'Antes Venus', 'Mars' opp 'Antes Mars', ' Venus' opp 'Antes Mars', 'Venus' opp 'Antes Venus', 'Antes Mars' opp 'Antes Venus')
#     If exist mirror aspect between natural and antes point without aspect between natural points: 
#       ('Mars' con 'Antes Chiron') and ('Chiron' con 'Antes Mars') 
#       required stay the houses ruler's aspect, if aspect has two houses rulers or any other points - stay any.
#     Antes aspect by own planet without natural points aspect ('Mars' con 'Antes Mars')
#     Remove mirror houses antes aspects: ('Asc' con 'Mars') - ('Dsc' con 'Antes Mars').
#     Remove one mirror 'Moon Node' aspect by houses with 'weak' status from 'den_point' planet or any.